In [54]:
import re
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
import os,sys
import tqdm

In [56]:
rootPath ='D:/git/IoT_Sensors_Security_Analysis/data/dataset/dfF/'
files = os.listdir(rootPath)

In [57]:
files

['delay_5379_0_2021-09-22_14_40_51.txt',
 'disorder_6523_0_2021-09-22_14_51_13.txt',
 'freeze_7626_0_2021-09-22_15_01_35.txt',
 'hop_8746_0_2021-09-22_15_11_57.txt',
 'mimic_9850_0_2021-09-22_15_22_19.txt',
 'noise_10951_0_2021-09-22_15_32_41.txt',
 'normal_11903_0_2021-09-22_15_41_46.txt',
 'normal_v2_13034_0_2021-09-22_15_52_08.txt',
 'repeat_14139_0_2021-09-22_16_02_29.txt',
 'spoof_15240_0_2021-09-22_16_12_51.txt']

In [58]:
malware = set([i.split('_')[0] for i in files])

In [59]:
malware

{'delay',
 'disorder',
 'freeze',
 'hop',
 'mimic',
 'noise',
 'normal',
 'repeat',
 'spoof'}

In [60]:
def get_systemcall_name_with_fork(line):
    syscall = ''
    linelist = line.split(' ')
    pid = int(linelist[0])
    time_cost = re.split('<|>', linelist[-1])

    try:
        time_cost = float(time_cost[1])
    except:
        time_cost = 0
    for i,l in enumerate(linelist):
        if re.match(r'\d{2}:\d{2}:\d{2}', l) is not None:
            timestamp = l
            if '(' in linelist[i+1]:
                syscall = linelist[i+1].split('(')[0]
            elif '<...' in linelist[i+1]:
                syscall = linelist[i+2]
            elif '+++' in linelist[i+1]:
                syscall = linelist[i+2]
            elif '---' in linelist[i+1]:
                syscall = linelist[i+2]
            else:
                syscall = '!!'+linelist[i+1]
            break
    return [pid, timestamp, syscall, time_cost]

In [61]:
def get_systemcall_name_without_fork(line):
    syscall = ''
    linelist = line.split(' ')
    # pid = int(linelist[0])
    time_cost = re.split('<|>', linelist[-1])

    try:
        time_cost = float(time_cost[1])
    except:
        time_cost = 0
    for i,l in enumerate(linelist):
        if re.match(r'\d{2}:\d{2}:\d{2}', l) is not None:
            timestamp = l
            if '(' in linelist[i+1]:
                syscall = linelist[i+1].split('(')[0]
            elif '<...' in linelist[i+1]:
                syscall = linelist[i+2]
            elif '+++' in linelist[i+1]:
                syscall = linelist[i+2]
            elif '---' in linelist[i+1]:
                syscall = linelist[i+2]
            else:
                syscall = '!!'+linelist[i+1]
            break
    return [timestamp, syscall, time_cost]

In [77]:
for ma in malware: 
    for fi in files:
        if ma in fi and 'txt' in fi:
            syscalls = []
            # par = tqdm.tqdm()
            with open(rootPath+fi,'r') as f:
                for line in f:
                    # par.update(1)
                    sys=get_systemcall_name_with_fork(line)
                    syscalls.append(sys)
                f.close()
                # par.close()
            df = pd.DataFrame(syscalls)
            df.columns=['pid', 'timestamp', 'syscall', 'time_cost']
            # sc = set(df['syscall'])
            ds = df['syscall'].describe()
            dt = df['time_cost'].describe()
            print('==============================================')
            print('Malware: {}'.format(ma))
            print('system call describe: {}, mean costed time: {}'.format(ds.to_dict(), dt['mean']))
            a = df.groupby('syscall')
            print(a.describe()[('pid','count')].to_dict())
            break



2849155it [00:30, 176393.15it/s]

Malware: delay
system call describe: {'count': 11755629, 'unique': 5, 'top': 'nanosleep', 'freq': 11754256}, mean costed time: 0.00021798508280586286
{'ioctl': 572.0, 'nanosleep': 11754256.0, 'poll': 240.0, 'timerfd_settime': 402.0, 'write': 159.0}
Malware: noise
system call describe: {'count': 10360695, 'unique': 16, 'top': 'nanosleep', 'freq': 10356691}, mean costed time: 0.0002643593029232111
{'SIGPIPE': 1.0, 'close': 16.0, 'exit': 12.0, 'exit_group': 1.0, 'exited': 9.0, 'futex': 14.0, 'getpid': 10.0, 'ioctl': 1824.0, 'madvise': 15.0, 'munmap': 4.0, 'nanosleep': 10356691.0, 'poll': 659.0, 'read': 4.0, 'sigreturn': 2.0, 'timerfd_settime': 1209.0, 'write': 224.0}
Malware: mimic
system call describe: {'count': 11796998, 'unique': 6, 'top': 'nanosleep', 'freq': 11795238}, mean costed time: 0.0002163828482466473
{'getpid': 4.0, 'ioctl': 751.0, 'nanosleep': 11795238.0, 'poll': 304.0, 'timerfd_settime': 520.0, 'write': 181.0}
Malware: disorder
system call describe: {'count': 11741706, 'uni

In [63]:
syscalls = []
tqdm
with open(rootPath+'delay_5379_0_2021-09-22_14_40_51.txt', 'r') as f:
    for line in f:
        sys=get_systemcall_name_with_fork(line)
        syscalls.append(sys)
    f.close()

# print('system call describe: {}'.format(dt.to_dict()))

In [64]:
df = pd.DataFrame(syscalls)
df.columns=['pid', 'timestamp', 'syscall', 'time_cost']
# sc = set(df['syscall'])
ds = df['syscall'].describe()
dt = df['time_cost'].describe()
print('==============================================')
print('Malware: {}'.format(ma))
print('system call describe: {}, mean costed time: {}'.format(ds.to_dict(), dt['mean']))

Malware: mimic
system call describe: {'count': 11755629, 'unique': 5, 'top': 'nanosleep', 'freq': 11754256}, mean costed time: 0.00021798508280586286


In [67]:
a = df.groupby('syscall')

In [72]:
d = a.describe()

In [75]:
d[('pid','count')].to_dict()

{'ioctl': 572.0,
 'nanosleep': 11754256.0,
 'poll': 240.0,
 'timerfd_settime': 402.0,
 'write': 159.0}

In [45]:
syscalls = []
par = tqdm.tqdm()
with open(rootPath+'disorder_6523_0_2021-09-22_14_51_13.txt', 'r') as f:
    for line in f:
        par.update(1)
        sys=get_systemcall_name_with_fork(line)
        syscalls.append(sys)
    f.close()
    par.close()

# print('system call describe: {}'.format(dt.to_dict()))

11741706it [01:10, 166183.72it/s]


In [46]:
df = pd.DataFrame(syscalls)
df.columns=['pid', 'timestamp', 'syscall', 'time_cost']
# sc = set(df['syscall'])
ds = df['syscall'].describe()
dt = df['time_cost'].describe()
print('==============================================')
print('Malware: {}'.format(ma))
print('system call describe: {}, mean costed time: {}'.format(ds.to_dict(), dt['mean']))

Malware: freeze
system call describe: {'count': 11741706, 'unique': 6, 'top': 'nanosleep', 'freq': 11740455}, mean costed time: 0.00022226212596363802


In [47]:
a = df.groupby('syscall')
a.describe()

pid                                                 \
                      count         mean       std     min     25%     50%   
syscall                                                                      
getpid                  4.0  6532.000000  0.000000  6532.0  6532.0  6532.0   
ioctl                 521.0  6526.000000  0.000000  6526.0  6526.0  6526.0   
nanosleep        11740455.0  6531.193333  0.765910  6523.0  6531.0  6531.0   
poll                  228.0  6525.995614  0.066227  6525.0  6526.0  6526.0   
timerfd_settime       366.0  6526.000000  0.000000  6526.0  6526.0  6526.0   
write                 132.0  6526.181818  1.032437  6526.0  6526.0  6526.0   

                                  time_cost                                \
                    75%     max       count      mean       std  min  25%   
syscall                                                                     
getpid           6532.0  6532.0         4.0  0.000034  0.000039  0.0  0.0   
ioctl            6526.0  6526.0       521.0  0.255832  1.189892  0.0  0.0   
nanosleep        6532.0  6532.0  11740455.0  0.000194  0.024387  0.0  0.0   
poll             6526.0  6526.0       228.0  0.467439  1.733857  0.0  0.0   
timerfd_settime  6526.0  6526.0       366.0  0.177814  0.962259  0.0  0.0   
write            6526.0  6532.0       132.0  0.217006  1.264234  0.0  0.0   

                                                
                      50%       75%        max  
syscall                                         
getpid           0.000034  0.000068   0.000069  
ioctl            0.000066  0.000264  10.100744  
nanosleep        0.000064  0.000117  28.144929  
poll             0.000058  0.002343  11.919106  
timerfd_settime  0.000061  0.000257   9.992403  
write            0.000056  0.000244  10.571421

In [1]:
import re
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
import os,sys
import tqdm

In [2]:
rootPath ='D:/git/IoT_Sensors_Security_Analysis/data/perf/'

In [57]:
dataPath = rootPath + 'data/'

In [62]:
datafiles = os.listdir(dataPath)
dfs = dict()
for fi in datafiles:
    df = pd.read_csv(dataPath+fi)
    dfs[dataPath+fi] = df
    break

In [59]:
def change_time(df):
    index= 0
    time = 0
    for index,time in enumerate(df['timestamp']):
        if time != 0:
            break
    for j in range(0,index):
        df.loc[j, 'timestamp'] = time
    return df

In [61]:
fi

'delay_1311_0_2021-09-24_12_04_26.txt'

In [65]:
df[0:3]

,pid,timestamp,syscall,time_cost
0,es_sensor/1314,93373.555,ioctl,0.000
1,es_sensor/1314,93373.555,timerfd_settime,0.000
2,es_sensor/1314,93373.555,ioctl,0.007


In [64]:
df[0:3]
df = change_time(df)
df

,pid,timestamp,syscall,time_cost
0,es_sensor/1314,93373.555,ioctl,0.000
1,es_sensor/1314,93373.555,timerfd_settime,0.000
2,es_sensor/1314,93373.555,ioctl,0.007
3,es_sensor/1314,93373.601,timerfd_settime,0.008
4,es_sensor/1314,93373.617,ioctl,0.042
...,...,...,...,...
1522376,es_sensor/1314,692701.415,ioctl,0.009
1522377,es_sensor/1314,692701.432,timerfd_settime,0.008
1522378,es_sensor/1314,692701.446,ioctl,0.006
1522379,es_sensor/1314,692701.465,timerfd_settime,0.008
